<a href="https://colab.research.google.com/github/AwaisAli37405/Deep_Learning/blob/master/DeepLearning_with_process_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>